- Pandas' `pd.merge()` and `join()` methods enable SQL-style joins.
- Facilitating complex data relationships and manipulations grounded in relational algebra. 
- These tools are essential for efficiently combining data from multiple sources.

``` python

DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=None, indicator=False, validate=None)
```

# 1. Categories of Joins in pd.merge()
Depending upon input data has cols with duplicate entries or not, merge method performs 3 types of joins:
- one to one join
- many to one join
- many to many join

It finds common columns in input dataframes and make them **key column** for merging.


In all it tries to preserve duplicate values of all columns, accordingly performs merging. 

## 1.1 One-to-One joins
- Simple like column join in pd.concat()

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
# Define df1
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})

df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [3]:
# Define df2
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

df2


,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [4]:
# Merge 
df3 = pd.merge(df1, df2)

df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


- **Same as pd.concat with axis 1.**
- See, in df1 and df2 employee order are different.
- And in merge result it follows df1 employee name order.
- Pandas considers here employee col as key and correctly account the order.

In [5]:
# Merge is same as concat with axis =1
df_concat = pd.concat([df1,df2], axis=1)
df_concat

,employee,group,employee,hire_date
0,Bob,Accounting,Lisa,2004
1,Jake,Engineering,Bob,2008
2,Lisa,Engineering,Jake,2012
3,Sue,HR,Sue,2014


## 1.2 Many-to-one joins
- When one column has duplicate entries.
- Like in in df3.group. 
- When we merge it with dataset with non-duplicate entries, it will preserve the duplicate entries of df3.
- And wherever necessary the other column is repeated based on duplicates in another column, see supervisor of merge(df3, df4).

In [6]:
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [7]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})

df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [8]:
print(pd.merge(df3, df4))

  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


## 1.3 Many-to-Many Joins

- When both left and right array cols have duplicates, then merge type is many to many join.



In [9]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux', 'spreadsheets', 'organization']})
df5


,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [10]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [11]:
pd.merge(df1, df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


- It repeated the non-duplicates duplicate cols to all possible duplicates.

# 2. Merge Key Specification

``` python
DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=None, indicator=False, validate=None)
```

- Column names in datasets do not match so neatly in real-world data.
- So, we use arguments of pd.merge() function.

## 2.1 'on' keyword

- Normally by default, merge function takes common column from both datasets as 'key column'.
- But we can provide the common column name using 'on' keyword to make that column key column for merge operation.

In [12]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [13]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


- Here, common column is employee, we can directly provide 'employee' column to make it key column.

In [14]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


- This merged the datasets on key column 'employee'.

## 2.2 'left_on' and 'right_on'  Keywords

- When column names is two datasets are different.
- e.g. one dataset has names of employees by col name 'employee' and other has 'name' column.
- Then we use these arguments.

In [15]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [16]:
# Create a new df3 with 'name' col name 
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
df3

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [17]:
# merge
pd.merge(df1, df3, left_on='employee', right_on='name')

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


- **There are two same columns in result, we can drop one of it using drop() method.**

## 2.3 'left_index' and 'right_index' Keywords
- To merge of index instead of columns.

In [18]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [19]:
# Let's make employee col as index col
df1a = df1.set_index('employee')
df1a


,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [20]:
df2a = df2.set_index('employee')
df2a

,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


In [21]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


## 2.4 Combine 'index' and 'on' arguments:

In [22]:
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [23]:
df3

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [24]:
pd.merge(df1a, df3, left_index=True, right_on='name')


,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


## 2.5 pd.join() : Merge with default indices

In [25]:
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [26]:
df2a

,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


- Defaul index are 'employee' for both.

In [27]:
df1a.join(df2a)   # Performed merge on default indices.

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


# 3. Specifying Set Arithmetic for Joins: 'how'



In [28]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df6


,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [29]:
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
df7

,name,drink
0,Mary,wine
1,Joseph,beer


In [30]:
pd.merge(df6, df7)   # Default inner merge

,name,food,drink
0,Mary,bread,wine


**pd.concat()**: 
- Mainly use for combining DataFrames along an axis (rows or columns). 
- Useful for simple stacking but less flexible for specific joins.
- Default join type is 'outer'.


**pd.merge()**: 
- Provides more control over how DataFrames are combined.
- Especially when you need to match on specific columns or indexes.
- Default how is 'inner'.

In [31]:
pd.merge(df6, df7, how='outer')

,name,food,drink
0,Joseph,NaN,beer
1,Mary,bread,wine
2,Paul,beans,NaN
3,Peter,fish,NaN


In [32]:
pd.merge(df6, df7, how= 'left')   # left join and right join return join over the left entries and right entries

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


- All of these options can be applied straightforwardly to any of the preceding join
types.

# 4. Overlapping Column Names: The suffixes Keyword



In [33]:
import pandas as pd

df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'rank': [3, 1, 4, 2]})


In [34]:
df8

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [35]:
df9

,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


- Here, ranks columns have conficting values.
- merge will automatically add suffix _x and _y.

In [36]:
pd.merge(df8, df9, on= 'name')

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


- We can give custome suffixes also.

In [37]:
pd.merge(df8, df9, on='name', suffixes=['_L', '_R'])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


# 5. Example: US States Data

- We will use data: 
  - state-population.csv
  - state-areas.csv
  - state-abbrevs.csv

- **To find: rank US states and territories by their 2010 population density.**

In [50]:
pop = pd.read_csv('state-population.csv')
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [51]:
areas = pd.read_csv('state-areas.csv')
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [52]:
abbrevs = pd.read_csv('state-abbrevs.csv')
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


**1. Combine data:**

In [53]:
# Merge pop and abbrevs

merged = pd.merge(pop, abbrevs, how= 'outer', left_on='state/region', right_on='abbreviation')
merged.head()

,state/region,ages,year,population,state,abbreviation
0,AK,total,1990,553290.0,Alaska,AK
1,AK,under18,1990,177502.0,Alaska,AK
2,AK,total,1992,588736.0,Alaska,AK
3,AK,under18,1991,182180.0,Alaska,AK
4,AK,under18,1992,184878.0,Alaska,AK


In [54]:
# Drop duplicate col: abbreviation

merged = merged.drop('abbreviation', axis=1) 
merged.head()

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


**2. Check for mismatch with null rows.**

In [55]:
# Check for mismatch with null rows.

merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

- There are nan in population, state, and abbreviation.
- Check which rows have nan values.

In [56]:
# Population col

merged[merged['population'].isnull()]

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN
1877,PR,under18,1993,NaN,NaN
1878,PR,under18,1992,NaN,NaN
1879,PR,total,1992,NaN,NaN
1880,PR,under18,1994,NaN,NaN
1881,PR,total,1994,NaN,NaN


   1. Missing population data
- PR state's some population data is missing.
- All the missing data are from 1999 and before.

   1. Missing state data
- Means in population table, some new entries were performed.
- Which were not updated in abbreviation table.

Therefore, now in full dataset, lets find which are the new state entries.
- These are the entries for which no state data is present.


In [57]:
merged[merged['state'].isnull()]['state/region'].unique()

array(['PR', 'USA'], dtype=object)

- The new entries are for PR and USA.
- These are not present in abbreviation table.
- So, let's fix this in merged table.

In [59]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'

In [63]:
merged[merged['state/region'] == 'PR'].head()

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,Puerto Rico
1873,PR,total,1990,NaN,Puerto Rico
1874,PR,total,1991,NaN,Puerto Rico
1875,PR,under18,1991,NaN,Puerto Rico
1876,PR,total,1993,NaN,Puerto Rico


In [75]:
merged.loc[merged['state/region']=='USA', 'state'] = 'United States'

In [79]:
merged[merged['state/region']=='USA'].head()

,state/region,ages,year,population,state
2160,USA,under18,1990,64218512.0,United States
2161,USA,total,1990,249622814.0,United States
2162,USA,total,1991,252980942.0,United States
2163,USA,under18,1991,65313018.0,United States
2164,USA,under18,1992,66509177.0,United States


- Check for nulls again.

In [81]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

- So no data on state column is missing now.
- Population data is missing, may be due to lack of data from original source?

**3. Combine area table with merged table**

In [89]:
merged.head()


,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


In [88]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [93]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


**4. Check for missing values**

In [95]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

- There are missing values in area col.
- Let's check.

In [111]:
final[final['area (sq. mi)'].isnull()]['state'].unique()

array(['United States'], dtype=object)

- All missing area data is for full USA.
- We will drop these values because for our problem its irrelevant.

In [113]:
final.dropna(inplace=True)

In [115]:
final.head()  # Cleaned data

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


**5. Get all data for year 2010**

In [136]:
final[final['year']==2010].head()

,state/region,ages,year,population,state,area (sq. mi)
42,AK,under18,2010,187902.0,Alaska,656425.0
43,AK,total,2010,713868.0,Alaska,656425.0
50,AL,under18,2010,1130966.0,Alabama,52423.0
51,AL,total,2010,4785570.0,Alabama,52423.0
140,AR,under18,2010,711947.0,Arkansas,53182.0


- We want total population data only.

In [137]:
total2010_data = final[(final['year']==2010) & (final['ages']=='total')]

In [138]:
total2010_data.head()

,state/region,ages,year,population,state,area (sq. mi)
43,AK,total,2010,713868.0,Alaska,656425.0
51,AL,total,2010,4785570.0,Alabama,52423.0
141,AR,total,2010,2922280.0,Arkansas,53182.0
149,AZ,total,2010,6408790.0,Arizona,114006.0
197,CA,total,2010,37333601.0,California,163707.0


- Or we could have used **query()**.

In [139]:
x = final.query("year == 2010 & ages == 'total'")
x.head()

,state/region,ages,year,population,state,area (sq. mi)
43,AK,total,2010,713868.0,Alaska,656425.0
51,AL,total,2010,4785570.0,Alabama,52423.0
141,AR,total,2010,2922280.0,Arkansas,53182.0
149,AZ,total,2010,6408790.0,Arizona,114006.0
197,CA,total,2010,37333601.0,California,163707.0


**6. Compute Population density.**

In [140]:
total2010_data= total2010_data.set_index('state')


In [141]:
total2010_data.head()

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alaska,AK,total,2010,713868.0,656425.0
Alabama,AL,total,2010,4785570.0,52423.0
Arkansas,AR,total,2010,2922280.0,53182.0
Arizona,AZ,total,2010,6408790.0,114006.0
California,CA,total,2010,37333601.0,163707.0


In [145]:
total2010_data['density'] = total2010_data['population']/total2010_data['area (sq. mi)']

In [147]:
total2010_data.head()

,state/region,ages,year,population,area (sq. mi),density
state,,,,,,
Alaska,AK,total,2010,713868.0,656425.0,1.087509
Alabama,AL,total,2010,4785570.0,52423.0,91.287603
Arkansas,AR,total,2010,2922280.0,53182.0,54.948667
Arizona,AZ,total,2010,6408790.0,114006.0,56.214497
California,CA,total,2010,37333601.0,163707.0,228.051342


**7. Rank states on density in ascending order.**

In [157]:
total2010_data.sort_values(ascending=False, inplace=True, by='density')

In [162]:
total2010_data.head(10)  # Top 10 with highest population density in 2010
                         # Highest is District of Columbia	i.e. Washington Dc

,state/region,ages,year,population,area (sq. mi),density
state,,,,,,
District of Columbia,DC,total,2010,605125.0,68.0,8898.897059
Puerto Rico,PR,total,2010,3721208.0,3515.0,1058.665149
New Jersey,NJ,total,2010,8802707.0,8722.0,1009.253268
Rhode Island,RI,total,2010,1052669.0,1545.0,681.339159
Connecticut,CT,total,2010,3579210.0,5544.0,645.600649
Massachusetts,MA,total,2010,6563263.0,10555.0,621.815538
Maryland,MD,total,2010,5787193.0,12407.0,466.445797
Delaware,DE,total,2010,899711.0,1954.0,460.445752
New York,NY,total,2010,19398228.0,54475.0,356.094135


In [161]:
total2010_data.tail()  # Lowest population density is Alaska

,state/region,ages,year,population,area (sq. mi),density
state,,,,,,
South Dakota,SD,total,2010,816211.0,77121.0,10.583512
North Dakota,ND,total,2010,674344.0,70704.0,9.537565
Montana,MT,total,2010,990527.0,147046.0,6.736171
Wyoming,WY,total,2010,564222.0,97818.0,5.768079
Alaska,AK,total,2010,713868.0,656425.0,1.087509
